<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 The MediaPipe Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with MediaPipe Model Maker

In this colab notebook, you'll learn how to use [MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker) to train a custom object detection model to detect dogs.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset to use with the [MediaPipe Object Detector task](https://developers.google.com/mediapipe/solutions/vision/object_detector).


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/google/mediapipe/tree/master/mediapipe/model_maker) and the pycocotools library you'll use for evaluation.

In [2]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [3]:
!pip install 'keras<3.0.0' mediapipe-model-maker

Import the required packages.

In [4]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from google.colab import files

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [5]:
import zipfile
from google.colab import drive

drive.mount('/content/gdrive/')
zip_ref = zipfile.ZipFile("/content/gdrive/MyDrive/rock-paper-scissors.zip", 'r')
!unzip "/content/gdrive/MyDrive/rock-paper-scissors.zip"
train_dataset_path = "rock-paper-scissors/train"
validation_dataset_path = "rock-paper-scissors/validate"

Mounted at /content/gdrive/
Archive:  /content/gdrive/MyDrive/rock-paper-scissors.zip
   creating: rock-paper-scissors/test/
   creating: rock-paper-scissors/test/Annotations/
  inflating: rock-paper-scissors/test/Annotations/label_0_20220818_110354_379__jpg.rf.4d3e0adc763d33d04f30838c5623f612.xml  
  inflating: rock-paper-scissors/test/Annotations/label_0_20220818_110401_357__jpg.rf.f0312e00536c4f289f8f3b4359c5780a.xml  
  inflating: rock-paper-scissors/test/Annotations/label_0_20220818_110407_441__jpg.rf.bb7b68191a683434212bc23e20430b20.xml  
  inflating: rock-paper-scissors/test/Annotations/label_0_20220818_110441_500__jpg.rf.e3199ef97fd21ce9c9d7f548a3353fc9.xml  
  inflating: rock-paper-scissors/test/Annotations/label_0_20220818_110444_020__jpg.rf.9cd8dbf648d1f490a89fdc8baf1298a4.xml  
  inflating: rock-paper-scissors/test/Annotations/label_0_20220818_110456_517__jpg.rf.d7ea36f14721f8f5aae712dbcafc76ec.xml  
  inflating: rock-paper-scissors/test/Annotations/label_0_20220818_110457_

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [7]:
train_data = object_detector.Dataset.from_pascal_voc_folder(
    'rock-paper-scissors/train',
    cache_dir="/tmp/od_data/train",
)

val_data = object_detector.Dataset.from_pascal_voc_folder(
    'rock-paper-scissors/test',
    cache_dir="/tmp/od_data/validatation")

### Step 2: Train the TensorFlow model with the training data.

* Set `batch_size = 8` here so you will see that it takes 13 steps to go through the 139 images in the training dataset.
* Set `learning_rate = 0.3` here so the model will adjust its weights at a rate of 0.3.
* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.


In [9]:
hparams = object_detector.HParams(batch_size=8, learning_rate=0.3, epochs=10, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=object_detector.SupportedModels.MOBILENET_V2,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

Using existing files at /tmp/model_maker/object_detector/mobilenetv2_i256
Model: "retina_net_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_1 (MobileNet)    {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn_1 (FPN)                 {'5': (None, 8, 8, 128)   149056    
                             , '4': (Non

27/27 [==============================] - 95s 2s/step - total_loss: 2.9691 - cls_loss: 2.6481 - box_loss: 0.0053 - model_loss: 2.9141 - val_total_loss: 1.3235 - val_cls_loss: 1.0757 - val_box_loss: 0.0039 - val_model_loss: 1.2685
Epoch 2/3
27/27 [==============================] - 59s 2s/step - total_loss: 1.0013 - cls_loss: 0.8566 - box_loss: 0.0018 - model_loss: 0.9462 - val_total_loss: 0.9350 - val_cls_loss: 0.7660 - val_box_loss: 0.0023 - val_model_loss: 0.8799
Epoch 3/3
27/27 [==============================] - 58s 2s/step - total_loss: 0.6344 - cls_loss: 0.5305 - box_loss: 9.7460e-04 - model_loss: 0.5792 - val_total_loss: 0.7512 - val_cls_loss: 0.6029 - val_box_loss: 0.0019 - val_model_loss: 0.6960


### Step 3. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 27 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [10]:
loss, coco_metrics = model.evaluate(val_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

15/15 [==============================] - 3s 190ms/step - total_loss: 0.7452 - cls_loss: 0.5927 - box_loss: 0.0019 - model_loss: 0.6900
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.18s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.396
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.302
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.259
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.547
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.723
 Average Recall     (AR) @

### Step 4: Export as a TensorFlow Lite model.

After creating the model, convert and export it to a Tensorflow Lite model format, and then download it for later use in an on-device application with MediaPipe Tasks.

In [11]:
model.export_model('rock-paper-scissors.tflite')
!ls exported_model
files.download('exported_model/rock-paper-scissors.tflite')

Exporting a floating point model


/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	metadata.json
float_ckpt.index		rock-paper-scissors.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>